# Link census tracts to TOC tiers
* Find what proportion belongs within a particular tier
* Make a crosswalk that tells you what % of that tract falls in what tier

In [1]:
import intake
import numpy as np
import pandas as pd
import geopandas as gpd
import boto3
import utils

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

s3 = boto3.client('s3')
bucket_name = 'city-planning-entitlements'

To do: figure out where toc_parcels_tracts.zip is created (probably in scripts)
What is the universe of tracts?
Tracts that touch TOC Tiers? Probably, but they should definitely have a 0, 1, 2, 3, 4 observation, then we can fill in the corresponding % for each tier, and it has to sum to 1. No more, no less.
That should be our crosswalk, so when we merge on GEOID and TOC_Tier, we will not be losing obs like the way we are now.

In [ ]:
toc_parcels = gpd.read_file(
    f"s3://{bucket_name}/gis/intermediate/toc_eligible_parcels_with_entitlements.geojson"
).to_crs({'init':'epsg:2229'})

crosswalk = gpd.read_file(
            f'zip+s3://{bucket_name}/gis/intermediate/toc_parcels_tracts.zip').to_crs({'init':'epsg:2229'})

tracts = catalog.census_tracts.read().to_crs({'init':'epsg:2229'})

tracts = (
    tracts[['GEOID10', 'geometry']]
    .rename(columns = {'GEOID10': 'GEOID'})
    .assign(tract_sqft = tracts.geometry.area)
    .drop(columns = 'geometry')
)

In [ ]:
# Merge in parcel-to-tracts crosswalk with those that have TOC entitlements
m1 = pd.merge(crosswalk, toc_parcels.drop(columns = 'geometry'), on = 'AIN', how = 'left', validate = '1:1')

# Merge in tract's geometry
m2 = pd.merge(m1, tracts, on = 'GEOID', how = 'inner', validate = 'm:1')

In [ ]:
drop_cols = ['x', 'y', 'num_AIN']

col_order = ['AIN', 'TOC_Tier', 'zone_class', 'num_TOC', 'num_nonTOC',
            'parcelsqft', 'geometry', 'GEOID', 'pop', 'parcel_tot', 'tract_sqft']

m3 = (m2
      .drop(columns = drop_cols)
      # Fill in info for parcels that don't have entitlements
      .assign(
          TOC_Tier = m2.TOC_Tier.fillna(0).astype(int),
          num_TOC = m2.num_TOC.fillna(0).astype(int),
          num_nonTOC = m2.num_nonTOC.fillna(0).astype(int),
      )
      .reindex(columns = col_order)
)

In [ ]:
m4 = m3.groupby(['GEOID', 'tract_sqft', 'parcel_tot', 'TOC_Tier']).agg({'parcelsqft':'sum'}).reset_index()

m4 = (m4.assign(
        pct = m4.parcelsqft / m4.parcel_tot,
    ).sort_values(['GEOID', 'pct'], ascending = [True, False])[
    ['GEOID', 'TOC_Tier', 'parcelsqft', 'pct']]
)

In [ ]:
m4.to_parquet('../data/crosswalk_tracts_tiers.parquet')
m4.to_parquet(f's3://{bucket_name}/data/crosswalk_tracts_tiers.parquet')

## crosswalk_tracts_tiers
We want to expand this so it has obs for tiers 0-4.

In [ ]:
crosswalk = pd.read_parquet(f's3://{bucket_name}/data/crosswalk_tracts_tiers.parquet')

In [ ]:
just_GEOID = pd.DataFrame(np.repeat(crosswalk[['GEOID']].drop_duplicates().values, 5, axis=0))
just_GEOID.columns = ['GEOID']
just_GEOID['TOC_Tier'] = just_GEOID.groupby('GEOID').cumcount()

In [ ]:
pd.merge(just_GEOID, crosswalk, how = 'left', validate = '1:1').sort_values(['GEOID', 'TOC_Tier'])

## toc_parcels_tracts